In [20]:
from deltalake import DeltaTable
import numpy as np
import pandas as pd

bets = DeltaTable("/Users/andreikirpichev/Downloads/bets/")
bets = bets.to_pandas()
bets['label'] = np.where(bets['stake_eur'] - bets['weekday_stake'] > 0, 1, 0)
bets['label'].value_counts()

0    3311494
1    1560154
Name: label, dtype: int64

In [27]:
bets.head()

,customer_id,age,country,section,point,market,market_params,contest_id,placement_date,placement_time,...,sport,league,minutes_before_start,stake_eur,dayofweek,weekday_stake,section_2w,point_2w,market_2w,label
0,9786810,26,2,37,7,12,2320,15531,2022-09-01,2022-09-01 00:00:07.176,...,13,618,60.0,11.25,5,2.98,6.25,6.25,6.56,1
1,9187810,26,2,39,6,7,2320,63387,2022-09-01,2022-09-01 00:00:20.216,...,28,1484,0.0,5.00,5,3.56,3.02,3.25,2.94,1
2,9187810,26,2,86,6,9,2320,63380,2022-09-01,2022-09-01 00:00:20.216,...,28,1481,0.0,5.00,5,3.58,5.00,3.25,3.26,1
3,12072510,44,2,10,7,12,2320,15484,2022-09-01,2022-09-01 00:00:22.536,...,13,919,0.0,3.75,5,3.29,3.75,3.75,3.75,1
4,12072510,44,2,37,7,12,2320,15476,2022-09-01,2022-09-01 00:00:22.536,...,13,415,0.0,3.75,5,3.26,3.75,3.75,3.75,1


In [21]:
from sklearn.preprocessing import LabelEncoder

categorical_features = ['country', 'section', 'point', 'market', 'market_params', 'contest_id', 'home', 'away', 'region', 'sport', 'league']

le = LabelEncoder()

# Convert the variables to numerical
for i in range(len(categorical_features)):
    label = le.fit_transform(bets[categorical_features[i]])
    bets[categorical_features[i]] = label
bets.head()


,customer_id,age,country,section,point,market,market_params,contest_id,placement_date,placement_time,...,sport,league,minutes_before_start,stake_eur,dayofweek,weekday_stake,section_2w,point_2w,market_2w,label
0,9786810,26,2,37,7,12,2320,15531,2022-09-01,2022-09-01 00:00:07.176,...,13,618,60.0,11.25,5,2.98,6.25,6.25,6.56,1
1,9187810,26,2,39,6,7,2320,63387,2022-09-01,2022-09-01 00:00:20.216,...,28,1484,0.0,5.00,5,3.56,3.02,3.25,2.94,1
2,9187810,26,2,86,6,9,2320,63380,2022-09-01,2022-09-01 00:00:20.216,...,28,1481,0.0,5.00,5,3.58,5.00,3.25,3.26,1
3,12072510,44,2,10,7,12,2320,15484,2022-09-01,2022-09-01 00:00:22.536,...,13,919,0.0,3.75,5,3.29,3.75,3.75,3.75,1
4,12072510,44,2,37,7,12,2320,15476,2022-09-01,2022-09-01 00:00:22.536,...,13,415,0.0,3.75,5,3.26,3.75,3.75,3.75,1


In [22]:
# Create features and target
XX = bets[['customer_id', 'age', 'country', 'section', 'point', 'market', 'market_params', 'contest_id', 'home', 'away', 'region', 'sport', 'league', 'minutes_before_start', 'weekday_stake', 'section_2w', 'point_2w', 'market_2w']]
X = bets[['age', 'country', 'section', 'point', 'market', 'region', 'sport', 'league', 'weekday_stake', 'section_2w', 'point_2w', 'market_2w']]
y = bets[['label']]

# show rows with NaN values
# X[X.isna().any(axis=1)]

# Make necessary imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 101, shuffle=False)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

LogisticRegression()

In [23]:
import warnings
warnings.filterwarnings('ignore')

# Make prediction
predictions = model.predict(X_test)

result = X_test
result['label'] = y_test
result['prediction'] = predictions.tolist()
result.head()

,age,country,section,point,market,region,sport,league,weekday_stake,section_2w,point_2w,market_2w,label,prediction
3653736,45,2,37,7,12,40,13,1161,0.35,0.28,0.28,0.28,1,0
3653737,45,2,37,7,12,33,13,175,0.35,0.28,0.28,0.28,1,0
3653738,45,2,37,7,12,18,13,1287,0.35,0.28,0.28,0.28,1,0
3653739,45,2,37,7,12,130,18,1011,0.35,0.28,0.28,0.28,1,0
3653740,45,2,37,7,12,41,13,837,0.34,0.28,0.28,0.28,1,0


In [24]:
# Import library for metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Print metrics
print("Accuracy:", accuracy_score(y_test, predictions))
print("Precision:", precision_score(y_test, predictions))
print("Recall:", recall_score(y_test, predictions))

Accuracy: 0.6849591760324227
Precision: 0.5428263214670982
Recall: 0.013087602669538033
